## Семинар 1
# Оптимизация гиперпараметров

Гиперпараметр (англ. hyperparameter) — параметр, который не настраивается во время обучения модели. Пример гиперпараметра — шаг градиентного спуска, он задается перед обучением. Пример параметров — веса градиентного спуска, они изменяются и настраиваются во время обучения.

Попытаемся улучшить обобщающую способность модели, настроив ее гиперпараметры.
Задача оптимизации гиперпараметров заключается в подборе таких гиперпараметров, при которых заданная модель алгоритма будет наиболее эффективна.
Наиболее часто используемый метод – это решетчатый поиск (grid search), который перебирает все возможные комбинации интересующих гиперпараметров.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Задания для выполнения:

### 1. Сгенерируйте данные для задачи бинарной классификации с 4 признаками и 10000 наблюдениями. Целевая переменная должна зависеть от 2 признаков. Выведите признаки в виде датафрейма, дав названия колонкам. Выведите число объектов в каждом классе.
Указание: Для этого воспользуйтесь [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html), задав параметры: число строк n_samples=10000, число признаков n_features=4, число признаков от которых зависит целевая переменная n_informative=2, число избыточных признаков (т.е. признаков, являющихся линейными комбинациями признаков от которых зависит целевая переменная) n_redundant=2, фиксируем воспроизводимость случайных данных random_state=42.

In [2]:
X,y = make_classification (n_samples=10000,
                          n_features=4,
                          n_informative=2,
                          n_redundant=2,
                          random_state=42)

In [3]:
df = pd.DataFrame(data=X, columns=['X'+str(i) for i in range(1, X.shape[1]+1)])
df.head(5)

,X1,X2,X3,X4
0,1.522650,-0.934560,-0.465022,0.058874
1,1.048103,-0.746806,0.436853,0.859628
2,1.024216,-0.449412,-1.623434,-1.378656
3,-0.434667,0.281754,0.023299,-0.135243
4,1.421165,-0.731574,-1.462926,-1.058435


In [4]:
pd.Series(y).value_counts()

0    5000
1    5000
dtype: int64

### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 20% объектов и зафиксируйте способ перемешивания данных random_state=42. Выведите размеры обучающей и тестовой частей.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [6]:
X_train.shape[0], X_test.shape[0]

(8000, 2000)

### 3. Постройте модель логистической регрессии с гиперпараметрами, установленными по умолчанию. Для этого создайте класс LogisticRegression c дефолтными гиперпараметрами и обучите модель на обучающих данных. Сделайте предсказание на тестовых данных. Вычислите метрику accuracy на тестовых данных.

In [7]:
default_model = LogisticRegression()
default_model.fit(X_train, y_train)

LogisticRegression()

In [8]:
default_model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [9]:
y_test_pred = default_model.predict(X_test)

In [10]:
default_accuracy = metrics.accuracy_score(y_test, y_test_pred)
default_accuracy

0.8935

### 4. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием hold-out разбиения, среди двух гиперпараметров, значения которых указанны ниже. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

Создадим диапазон вариантов для гиперпараметров в виде словаря:

In [11]:
param_grid = {'C': np.logspace(-3,2,9), 
              'solver': ['lbfgs','liblinear','saga']}

In [12]:
def Grid(X_train, y_train):
    X_trainval, X_testval, y_trainval, y_testval = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C=C, solver=solver)
            model.fit(X_trainval, y_trainval)
            score = model.score(X_testval, y_testval)
            if score > best_score:
                best_score = score
                best_parameters = {'C':C, 'solver':solver}
    return best_parameters

In [13]:
best_parameters = Grid(X_train, y_train)
best_parameters

{'C': 0.004216965034285823, 'solver': 'lbfgs'}

In [14]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)

0.894

### 5. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

In [15]:
def GridCV(X_train, y_train):
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C=C, solver=solver)
            score = np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))
            if score > best_score:
                best_score = score
                best_parameters = {'C':C, 'solver':solver}
    return best_parameters

In [16]:
best_parameters = GridCV(X_train, y_train)
best_parameters

{'C': 0.01778279410038923, 'solver': 'lbfgs'}

In [17]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
best_model.score(X_test, y_test)

0.894

### 6. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании, используя класс GridSearchCV из библиотеки sklearn. Убедитесь, что результаты будут теми же, что и в предыдущем задании.
Указание: Для выведения наилучших гиперпараметров модели, используйте атрибут best_params_. Наилучшая модель выводится при помощи атрибута best_estimator_.

In [18]:
gs = GridSearchCV(
        estimator = LogisticRegression(),
        param_grid = param_grid,
        scoring = 'accuracy',
        cv = 5)

In [19]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [20]:
gs.best_params_

{'C': 0.01778279410038923, 'solver': 'lbfgs'}

In [21]:
best_model_skl = LogisticRegression(**gs.best_params_)
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.894

или

In [22]:
best_model_skl = gs.best_estimator_
best_model_skl.fit(X_train, y_train)
best_model_skl.score(X_test, y_test)

0.894

### 7. Выведите результаты решетчатого поиска с использованием класса GridSearchCV, осуществлённого в предыдущем пункте, в виде словаря и в виде датафрейма. Оставьте в датафрейме только два столбца со значениями гиперпараметров и со значениями вычисляемой метрики. Выведите самое большое значение среди средних значений метрики accuracy перекрестной проверки.
Указание: Для выведения результатов решетчатого поиска в виде словаря используйте атрибут cv_results_. Для выведения самого большого значения среди средних значений метрики accuracy используйте атрибут best_score_.

In [23]:
gs.cv_results_

{'mean_fit_time': array([0.00490365, 0.00364184, 0.00708084, 0.00441885, 0.00381794,
        0.00654259, 0.00384827, 0.00381274, 0.00683351, 0.00380988,
        0.00397811, 0.00823212, 0.00508132, 0.00507307, 0.00637078,
        0.00390267, 0.00412102, 0.00695353, 0.00386124, 0.00425777,
        0.00671444, 0.00393195, 0.00421038, 0.00668693, 0.00384855,
        0.00423737, 0.00721354]),
 'std_fit_time': array([4.30391819e-04, 2.37651210e-04, 6.49035165e-04, 5.02638389e-04,
        1.96768377e-04, 6.97887511e-04, 1.86448414e-04, 2.16075137e-04,
        7.05072735e-04, 3.94872994e-05, 2.25747553e-04, 7.44291651e-04,
        9.00574514e-04, 3.77881910e-04, 3.59724448e-04, 1.73274186e-04,
        2.61459141e-04, 3.13299264e-04, 1.04182470e-04, 2.71341192e-04,
        3.86290623e-04, 1.87601543e-04, 1.91226956e-04, 6.49895863e-04,
        1.47126249e-04, 2.05739259e-04, 1.29698471e-03]),
 'mean_score_time': array([0.0007514 , 0.0006299 , 0.00053978, 0.0006568 , 0.00054364,
        0.000506

In [24]:
pd.DataFrame(gs.cv_results_).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004904,0.000430,0.000751,0.000082,0.001,lbfgs,"{'C': 0.001, 'solver': 'lbfgs'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
1,0.003642,0.000238,0.000630,0.000035,0.001,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.88375,0.878750,0.887500,0.891875,0.895000,0.887375,0.005761,25
2,0.007081,0.000649,0.000540,0.000044,0.001,saga,"{'C': 0.001, 'solver': 'saga'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
3,0.004419,0.000503,0.000657,0.000117,0.004217,lbfgs,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.88625,0.881875,0.891875,0.900000,0.896875,0.891375,0.006654,8
4,0.003818,0.000197,0.000544,0.000025,0.004217,liblinear,"{'C': 0.004216965034285823, 'solver': 'libline...",0.88750,0.883125,0.891875,0.900625,0.896250,0.891875,0.006187,4


In [25]:
pd.DataFrame({'params':gs.cv_results_['params'], 'mean_test_score':gs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'C': 0.001, 'solver': 'lbfgs'}",0.887125
1,"{'C': 0.001, 'solver': 'liblinear'}",0.887375
2,"{'C': 0.001, 'solver': 'saga'}",0.887125
3,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.891375
4,"{'C': 0.004216965034285823, 'solver': 'libline...",0.891875
5,"{'C': 0.004216965034285823, 'solver': 'saga'}",0.891375
6,"{'C': 0.01778279410038923, 'solver': 'lbfgs'}",0.892500
7,"{'C': 0.01778279410038923, 'solver': 'liblinear'}",0.892250
8,"{'C': 0.01778279410038923, 'solver': 'saga'}",0.892500
9,"{'C': 0.07498942093324558, 'solver': 'lbfgs'}",0.891750


In [26]:
gs.best_score_ # на обучающих данных

0.8924999999999998

### 8. Используйте методы score и predict класса GridSearchCV, чтобы вычислить метрики качества accuracy и F1-меры на тестовых данных для модели LogisticRegression с лучшими гиперпараметрами.

In [27]:
accuracy = gs.score(X_test, y_test)
accuracy

0.894

In [28]:
y_test_pred = gs.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred)
f1

0.8922764227642276

### 9. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс GridSearchCV из библиотеки sklearn, среди списка словарей гиперпараметров, указанных ниже. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [29]:
param_grid = param_grid = [
    {'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]},
    {'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]}]

In [30]:
gs_list = GridSearchCV(
        estimator = LogisticRegression(),
        param_grid = param_grid,
        scoring = 'accuracy',
        cv = 5)

In [31]:
gs_list.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                          'max_iter': [1000], 'penalty': ['l1', 'l2'],
                          'solver': ['liblinear']},
                         {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                          'max_iter': [1000], 'penalty': ['elasticnet'],
                          'solver': ['saga']}],
             scoring='accuracy')

In [32]:
gs_list.best_params_

{'C': 0.01778279410038923,
 'max_iter': 1000,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [33]:
accuracy = gs_list.score(X_test, y_test)
accuracy

0.894

In [34]:
y_test_pred = gs_list.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred)
f1

0.8928210313447926

### 10. Осуществите рандомизированный решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс RandomizedSearchCV из библиотеки sklearn, для списка словарей гиперпараметров из предыдущего задания. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [35]:
rgs = RandomizedSearchCV(
        estimator = LogisticRegression(),
        param_distributions = param_grid,
        scoring = 'accuracy',
        cv = 5,
        random_state = 1)

In [36]:
rgs.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions=[{'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                                         'max_iter': [1000],
                                         'penalty': ['l1', 'l2'],
                                         'solver': ['liblinear']},
                                        {'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
                                         'max_iter': [1000],
                                         'penalty': ['elasticnet'],
                                         'solver': ['saga']}],
                   random_state=1, scoring='accuracy')

In [37]:
gs_list.best_params_

{'C': 0.01778279410038923,
 'max_iter': 1000,
 'penalty': 'l1',
 'solver': 'liblinear'}

In [38]:
pd.DataFrame({'params':rgs.cv_results_['params'], 'mean_test_score':rgs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
1,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
2,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.891875
3,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
4,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
5,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890875
6,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890750
7,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.893250
8,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.888750
9,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890875


In [39]:
accuracy = gs_list.score(X_test, y_test)
accuracy

0.894

In [40]:
y_test_pred = gs_list.predict(X_test)
f1 = metrics.f1_score(y_test, y_test_pred)
f1

0.8928210313447926

# Задания для самостоятельного выполнения:

### 1. Загрузите встроенный датасет `load_iris`. Обозначьте признаки за `X`, а целевую переменную за `y`. Выведите первые 5 строк в виде датафрейма. 

In [41]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [42]:
pd.DataFrame(data=iris.data, columns=iris.feature_names)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 15% объектов и зафиксируйте способ перемешивания данных random_state=0. Выведите размеры обучающей и тестовой частей.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

In [44]:
X_train.shape[0], X_test.shape[0]

(127, 23)

### 3. Для датасета load_iris осуществите классификацию методом логистической регрессии, создав класс LogisticRegression(max_iter = 1000). Выведите значения гиперпараметров этой модели, установленные по умолчанию. Обучите эту модель, сделайте предсказание. Оцените качество модели, выведя значения метрик accuracy и f1-score.

In [45]:
model = LogisticRegression(max_iter=1000)
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [46]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred, average='macro')
accuracy, f1

(1.0, 1.0)

### 4. Оптимизируйте гиперпараметры, используя следующие наборы гиперпараметров: param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},] по метрике accuracy. Реализуйте поиск лучших гиперпараметров с перекрёстной проверкой, используя класс GridSearchCV. Выведете результаты поиска в виде датафрейма, состоящего из двух колонок: в первой колонке все перебираемые сочитания гиперпараметров, а во второй соответствующие им средние значения метрики accuracy. Выведите значения лучших гиперпараметров.

In [47]:
param_grid = [
    {'solver': ['lbfgs', 'newton-cg'], 'penalty': ['l2']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
    {'C': np.logspace(0, 4, 10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']}]

gs = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid,
    scoring='accuracy',
    cv=5)

gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
             param_grid=[{'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2', 'l1'], 'solver': ['liblinear']},
                         {'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg']}],
             scoring='accuracy')

In [48]:
pd.DataFrame({'params':rgs.cv_results_['params'], 'mean_test_score':rgs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
1,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
2,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.891875
3,"{'solver': 'liblinear', 'penalty': 'l2', 'max_...",0.890875
4,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890750
5,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890875
6,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.890750
7,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.893250
8,"{'solver': 'liblinear', 'penalty': 'l1', 'max_...",0.888750
9,"{'solver': 'saga', 'penalty': 'elasticnet', 'm...",0.890875


In [49]:
gs.best_params_

{'C': 21.544346900318832, 'penalty': 'l1', 'solver': 'liblinear'}

### 5. Выведите значение метрики accuracy на тестовых данных. Cделайте предсказание для лучшей модели и выведите метрику f1-score. Сравните полученные метрики с метриками, полученными без оптимизации гиперпараметров.

In [50]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
f1 = metrics.f1_score(y_test, y_pred, average='weighted')
accuracy, f1

(0.9565217391304348, 0.9550514597773576)

### 6. Осуществите рандомизированный решетчатый поиск лучших гиперпараметров по метрике accuracy для той же самой сетки гиперпараметров. 

In [51]:
rgs_accuracy = RandomizedSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_distributions=param_grid,
    scoring='accuracy',
    cv=5,
    random_state=42)

rgs_accuracy.fit(X_train, y_train)
rgs_accuracy.best_params_

{'solver': 'liblinear', 'penalty': 'l1', 'C': 166.81005372000593}

### 7. Осуществите решетчатый поиск лучших гиперпараметров по метрике f1 для той же самой сетки гиперпараметров. 

In [52]:
rgs_f1 = GridSearchCV(
    estimator=LogisticRegression(max_iter=1000),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5)

rgs_f1.fit(X_train, y_train)
rgs_f1.best_params_

{'C': 21.544346900318832, 'penalty': 'l1', 'solver': 'liblinear'}